<a href="https://colab.research.google.com/github/Kibet-Rotich/pneumonia-detection/blob/master/Pneumonia_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

doctors or other experts in hospitals are usually required to examine xray slides to diagnose patients like pneumonia patients. this process is usually error prone and requires doctors to use valuable time to diagnose even healthy exrays, and with the current shortage of doctors in kenya, everytime saved can be used by the doctor to serve patients better elsewhere.

this solution is a pneumonia predictor, where you can feed in the image of a chest exray and its going to predict the chances of the patient having pneumonia.
 the solution should act as an auxilliary tool, ruling out the obvious healthy xrays and ensuring the doctor only uses their time todiagnose sick patients.

 it also acts as a proof of concept that computer vision can be used to detect any diseases that can be seen through an xray ctscan or mri scan.

data, the data we have are xray images from normal scans and from pneumonia patients. the dataset has images categorized like this.
test normal 234 pneumonia 390
train normal 1341 pneumonia 3875
val normal 8 pneumonia 8


preprocessing, we are going to augment the images to ensure model doesnt overfit and the distribution is more equal, its also going to generate more photos which is good.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Path to your dataset folder
data_dir = '/content/drive/MyDrive/datasets'

# Check if it worked
print(os.listdir(data_dir))


Mounted at /content/drive
['Pneumonia prediction', 'pneumonia_cnn_model.h5']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Base directory
base_dir = "/content/drive/MyDrive/datasets/Pneumonia prediction"

train_dir = base_dir + "/train"
val_dir   = base_dir + "/val"
test_dir  = base_dir + "/test"

# Parameters
img_size = (150, 150)  # You can try (224,224) if using transfer learning later
batch_size = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Validation & test data should NOT be augmented
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from directories
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 1661s 10s/step - accuracy: 0.7442 - loss: 0.5799 - val_accuracy: 0.7500 - val_loss: 0.6469
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.8699 - loss: 0.3147 - val_accuracy: 0.9375 - val_loss: 0.4793
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.8964 - loss: 0.2507 - val_accuracy: 0.8125 - val_loss: 0.6071
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 325s 2s/step - accuracy: 0.9019 - loss: 0.2364 - val_accuracy: 0.7500 - val_loss: 0.6955
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.9224 - loss: 0.1906 - val_accuracy: 0.6875 - val_loss: 1.1625
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.9359 - loss: 0.1751 - val_accuracy: 0.9375 - val_loss: 0.2703
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.9253 - loss: 0.1821 - val_accuracy: 0.8125 - val_loss: 0.4703
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 0.9418 - loss: 0.1581 - val_ac

In [ ]:
test_loss, test_acc = model.evaluate(test_gen)
print("Test Accuracy:", test_acc)


20/20 ━━━━━━━━━━━━━━━━━━━━ 211s 11s/step - accuracy: 0.6307 - loss: 0.9137
Test Accuracy: 0.7964743375778198


In [ ]:
# Save model
model.save("/content/drive/MyDrive/datasets/pneumonia_cnn_model.h5")




In [ ]:
model.save('pneumonia_resnet_model.keras')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras import layers

# Base parameters
img_size = (224, 224)   # ResNet50 expects 224x224
batch_size = 32

# Use the same data generators but resize to 224x224
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Load pre-trained ResNet50 without the top layer
base_model = ResNet50(weights='imagenet',
                      include_top=False,
                      input_shape=(224,224,3))

# Freeze the base model layers (so they don’t retrain initially)
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = layers.Flatten()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train only the top layers
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=5  # try more (10–15) if GPU allows
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_gen)
print("Test Accuracy:", test_acc)


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 969s 6s/step - accuracy: 0.7156 - loss: 1.6146 - val_accuracy: 0.5000 - val_loss: 0.7079
Epoch 2/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 139s 732ms/step - accuracy: 0.7489 - loss: 0.5201 - val_accuracy: 0.5000 - val_loss: 0.7875
Epoch 3/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 721ms/step - accuracy: 0.7510 - loss: 0.4904 - val_accuracy: 0.5000 - val_loss: 0.6412
Epoch 4/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 720ms/step - accuracy: 0.7402 - loss: 0.4626 - val_accuracy: 0.5000 - val_loss: 0.7362
Epoch 5/5
163/163 ━━━━━━━━━━━━━━━━━━━━ 117s 720ms/step - accuracy: 0.7391 - loss: 0.5192 - val_accuracy: 0.5000 - val_loss: 0.8505
20/20 ━━━━━━━━━━━━━━━━━━━━ 128s 7s/step - accuracy: 0.2984 - loss: 0.8606
Test Accuracy: 0.625


In [ ]:
# Unfreeze last few layers of ResNet
for layer in base_model.layers[-30:]:  # last 30 layers
    layer.trainable = True

# Compile again with a lower learning rate
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Continue training
history_finetune = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20
)

model.save("/content/drive/MyDrive/datasets/pneumonia_resnet_finetuned 2.keras")
20

Epoch 1/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 141s 760ms/step - accuracy: 0.9356 - loss: 0.1702 - val_accuracy: 0.6875 - val_loss: 1.1808
Epoch 2/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 119s 729ms/step - accuracy: 0.9386 - loss: 0.1732 - val_accuracy: 0.5625 - val_loss: 1.5546
Epoch 3/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 725ms/step - accuracy: 0.9343 - loss: 0.1685 - val_accuracy: 0.8125 - val_loss: 0.3636
Epoch 4/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 143s 732ms/step - accuracy: 0.9360 - loss: 0.1525 - val_accuracy: 0.5625 - val_loss: 4.3266
Epoch 5/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 120s 735ms/step - accuracy: 0.9377 - loss: 0.1652 - val_accuracy: 0.8125 - val_loss: 0.6380
Epoch 6/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 722ms/step - accuracy: 0.9423 - loss: 0.1412 - val_accuracy: 0.8125 - val_loss: 0.5120
Epoch 7/20
163/163 ━━━━━━━━━━━━━━━━━━━━ 118s 726ms/step - accuracy: 0.9398 - loss: 0.1481 - val_accuracy: 0.7500 - val_loss: 0.3932
Epoch 8/20
 60/163 ━━━━━━━━━━━━━━━━━━━━ 1:13 717ms/step - accuracy: 0.9447 -

In [ ]:
model.save("/content/drive/MyDrive/datasets/pneumonia_resnet_finetuned.h5")


In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(test_gen)
print("Test Accuracy:", test_acc)

20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 345ms/step - accuracy: 0.8639 - loss: 0.3376
Test Accuracy: 0.8573718070983887
